In [10]:
import torch
import pandas as pd
import numpy as np
import chromadb

In [11]:
FILES = !ls test_data

In [12]:
FILES[1]

'shard_100_11268.pt'

In [13]:
client = chromadb.PersistentClient(path="chroma/test.db")

In [14]:
collection = client.get_or_create_collection(name="shards")

In [15]:
an_to_date = pd.read_parquet('an_to_date.parquet')
an_to_date['julian_date'] = an_to_date['date'].apply(lambda x: x.to_julian_date())
an_to_date.to_parquet('an_to_date.parquet')

In [7]:
an_to_date.head()

,date,an,julian_date
0,2016-07-29 14:03:20,AJAZEN0020160729ec7t000p1,2.457599e+06
1,2016-10-02 05:01:18,AJAZEN0020161002eca100009,2.457664e+06
2,2016-10-21 15:06:12,AJAZEN0020161021ecal000ru,2.457683e+06
3,2016-10-11 14:05:04,AJAZEN0020161011ecab000uo,2.457673e+06
4,2016-09-09 08:06:17,AJAZEN0020160909ec99000dx,2.457641e+06


In [16]:
an_to_date = pd.read_parquet('an_to_date.parquet')

for shard_name in FILES:
    shard = torch.load(f'test_data/{shard_name}')
    
    print(shard_name)
 
    documents = [i['text'] for i in shard]
    embeddings = [i['bert_embed_pool'].tolist()[0] for i in shard]
    ids = [str(i['id']) for i in shard]
    metadata = [{'an':i['an'], 
                 'dyad_name':i['dyad_name'], 
                 'pred_dyad':i['pred_dyad'][0],
                 'pred_proba':i['pred_proba'],
                 'date':str(an_to_date.loc[i['id']].date),
                 'jdate':an_to_date.loc[i['id']].julian_date} 
                for i in shard]
    collection.add(embeddings=embeddings, metadatas=metadata, documents=documents, ids=ids)

shard_0_11268.pt
shard_100_11268.pt
shard_101_11268.pt
shard_102_11268.pt
shard_103_11268.pt
shard_104_11268.pt
shard_105_11268.pt
shard_106_11268.pt
shard_107_11268.pt
shard_108_11268.pt
shard_109_11268.pt
shard_10_11268.pt
shard_110_11268.pt
shard_111_11268.pt
shard_112_11268.pt
shard_113_11268.pt
shard_114_11268.pt
shard_115_11268.pt
shard_116_11268.pt
shard_117_11268.pt
shard_118_11268.pt
shard_119_11268.pt
shard_11_11268.pt
shard_120_11268.pt
shard_121_11268.pt
shard_122_11268.pt
shard_123_11268.pt
shard_124_11268.pt
shard_125_11268.pt
shard_126_11268.pt
shard_127_11268.pt
shard_128_11268.pt
shard_129_11268.pt
shard_12_11268.pt
shard_130_11268.pt
shard_131_11268.pt
shard_132_11268.pt
shard_133_11268.pt
shard_134_11268.pt
shard_135_11268.pt
shard_136_11268.pt
shard_137_11268.pt
shard_138_11268.pt
shard_139_11268.pt
shard_13_11268.pt
shard_140_11268.pt
shard_141_11268.pt
shard_142_11268.pt
shard_143_11268.pt
shard_144_11268.pt
shard_145_11268.pt
shard_146_11268.pt
shard_147_11268.pt

In [17]:
pos_shard = [i for i in shard if len(i['dyad_name'])>1]

In [19]:
q_art = pos_shard[19]

In [20]:
q_art

{'id': 91137,
 'an': 'AFPR000020190208ef2800cnl',
 'dyad_id': 15515.0,
 'date': '2019-02-07 00:00:00.000',
 'ged_id': 279182.0,
 'pred_dyad': ('ASWJ - Civilians', 0.7710747122764587),
 'pred_proba': 0.7710747122764587,
 'text': 'Suspected jihadists kill 7 in north Mozambique Suspected jihadists have killed seven men and abducted four women in northern Mozambique in the latest violence to hit the Cabo Delgado region, local sources said Friday.\n\nThe bodies, which were cut into pieces, were left in Piqueue village, a local traditional leader told AFP, speaking on condition of anonymity. "People were surprised while sleeping in the forest," he said, indicating that the residents had fled the village for fear of being attacked.\n\nThe attackers also kidnapped four women, he said.\n\nMozambican police declined to comment, but a local army commander confirmed the attack.\n\n"We urge people to stay in villages where they have protection from the police and the military," he said.\n\nSince Oc

In [21]:
cur_jdate = an_to_date.loc[q_art['id']].julian_date

In [ ]:
q_art

In [23]:
# Get the same contextual stuff.

collection.query(q_art['bert_embed_pool'].tolist()[0], 
                 n_results=5,
                 where = {"$and": [
                     {"pred_dyad": {'$eq': q_art['dyad_name']}},
                     {"dyad_name": {'$eq': ''}},
                     {"jdate": {'$gte': cur_jdate-90}},
                     {"jdate": {'$lte': cur_jdate}}
                 ]})

{'ids': [['2009664', '841238', '1330172', '1504799', '1726469']],
 'distances': [[5.009353160858154,
   11.754509925842285,
   25.997045516967773,
   40.53388214111328,
   47.882781982421875]],
 'metadatas': [[{'an': 'BBCAP00020181213eecd0002t',
    'date': '2018-12-13 05:31:55',
    'dyad_name': '',
    'jdate': 2458465.7304976853,
    'pred_dyad': 'ASWJ - Civilians',
    'pred_proba': 0.7926105260848999},
   {'an': 'AFNWS00020181120eebk0015x',
    'date': '2018-11-20 14:00:25',
    'dyad_name': '',
    'jdate': 2458443.083622685,
    'pred_dyad': 'ASWJ - Civilians',
    'pred_proba': 0.9042006731033325},
   {'an': 'BBCAP00020190118ef1i000m9',
    'date': '2019-01-18 08:54:42',
    'dyad_name': '',
    'jdate': 2458501.8713194444,
    'pred_dyad': 'ASWJ - Civilians',
    'pred_proba': 0.8980265855789185},
   {'an': 'BBCAP00020181224eeco001rx',
    'date': '2018-12-24 19:09:26',
    'dyad_name': '',
    'jdate': 2458477.2982175928,
    'pred_dyad': 'ASWJ - Civilians',
    'pred_proba':

In [24]:
collection.query(q_art['bert_embed_pool'].tolist()[0], 
                 n_results=5,
                 where = {"$and": [
                     {"dyad_name": {'$eq': ''}},
                     {"jdate": {'$gte': cur_jdate-90}},
                     {"jdate": {'$lte': cur_jdate}}
                 ]})

{'ids': [['2009664', '841238', '1330172', '394467', '1246799']],
 'distances': [[5.009353160858154,
   11.754509925842285,
   25.997045516967773,
   32.375701904296875,
   37.171295166015625]],
 'metadatas': [[{'an': 'BBCAP00020181213eecd0002t',
    'date': '2018-12-13 05:31:55',
    'dyad_name': '',
    'jdate': 2458465.7304976853,
    'pred_dyad': 'ASWJ - Civilians',
    'pred_proba': 0.7926105260848999},
   {'an': 'AFNWS00020181120eebk0015x',
    'date': '2018-11-20 14:00:25',
    'dyad_name': '',
    'jdate': 2458443.083622685,
    'pred_dyad': 'ASWJ - Civilians',
    'pred_proba': 0.9042006731033325},
   {'an': 'BBCAP00020190118ef1i000m9',
    'date': '2019-01-18 08:54:42',
    'dyad_name': '',
    'jdate': 2458501.8713194444,
    'pred_dyad': 'ASWJ - Civilians',
    'pred_proba': 0.8980265855789185},
   {'an': 'AFPR000020190129ef1t00hld',
    'date': '2019-01-29 16:52:34',
    'dyad_name': '',
    'jdate': 2458513.2031712965,
    'pred_dyad': 'Government of Mozambique - IS',
    

In [26]:
!open .